In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../../')
print(os.getcwd())
from v2.strategy.indicators.optimal_v2 import Optimal_v2
from v2.model import Trading
from v2.strategy.indicators.param import Param
from v2.strategy.indicators.roc import RateOfChange
from v2.research.scripts.notebook_utils import notebookUtils
from v2.research.scripts.scoreboard import updateScoreboard
from sklearn.model_selection import train_test_split
import multiprocessing as mp
from itertools import repeat

#tensorflow stuff
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

from v2.strategy.strategies.atlas.atlas_v1_2 import Atlas_v1_2
from v2.strategy.strategies.atlas.atlas_v0_0 import Atlas_v0_0

/Users/rosscopeland/Desktop/personal/code/vivaldi/back_testing


In [2]:
nu = notebookUtils()
model_name = "gamemaker"

In [3]:
datasets, features, indicators = nu.loadData(indicators=[('SMA','close',''),('SMA','close','for_short'),('roc','SMA_for_short', 'shorter')],
                            param_spec={
                                'SMA':{'period':150}, 
                                'SMA_for_short':{'period':30}, 
                                'RateOfChange_shorter':{'period':45},
                                },
                            spans=[],
                            seperate_by_coin=True)

Loading data from XTZ...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from MATIC...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from IOTA...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from chunk 5...
Loading data from chunk 6...
Loading data from chunk 7...
Loading data from ONT...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from chunk 5...
Loading data from chunk 6...
Loading data from chunk 7...
Loading data from ETC...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Lo

In [22]:
dataset = nu.multiProcessSimulateData(datasets, Atlas_v0_0)

buys = dataset[dataset['is_potential_buy'] == True]
average_profit = sum(buys['simul_profit']) / len(buys['simul_profit'])
average_profit

0.0052508098973872465

In [23]:
model_dataset, features, indicators = nu.loadData(indicators=[('moh', 'time', ''), ('tod', 'time', ''), ('dow', 'time', ''), ('bop', 'close', '')],
                            param_spec={},
                            optimal_threshold={'buy':(0.005, 0.03)},
                            spans=[{'indicator_name':'willr',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [10,30]},
                                    {'indicator_name':'cci',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [10,30,60]},
                                    {'indicator_name':'rsi',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [2,3,5,10,30]},
                                    {'indicator_name':'natr',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [30,60,120,180]},
                                    {'indicator_name':'cmo',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [2,3,5,10,60]}
                                    ],
                            scale='minmaxwindow')

Loading data from XTZ...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from MATIC...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from IOTA...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from chunk 5...
Loading data from chunk 6...
Loading data from chunk 7...
Loading data from ONT...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from chunk 5...
Loading data from chunk 6...
Loading data from chunk 7...
Loading data from ETC...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Lo

In [24]:
joined_dataset = model_dataset.merge(dataset, on=['time', 'open', 'high', 'low', 'close', 'volume'])
joined_dataset = joined_dataset[joined_dataset['is_potential_buy'] == True]

joined_dataset.drop(["time", "open", "high", "volume", "low", "SMA", "SMA_for_short", "RateOfChange_shorter", "is_potential_buy"], axis=1, inplace=True)

joined_dataset.dropna(inplace=True)


In [25]:
joined_dataset["optimal"] = joined_dataset.apply(lambda x: nu.filter_optimal(x.simul_profit, (average_profit, 1.0), 'buy'),  axis=1)
joined_dataset.drop(["simul_profit"], axis=1, inplace=True)

In [26]:
trainX, testX, trainy, testy = nu.splitData(joined_dataset, split_size=0.2, y_column_name="optimal", shuffle_data=False, balance_unbalanced_data=True, balance_info={'multiplier_val':1, 'superset_class_val':0, 'randomize_concat':True})
len(trainX)

172121

In [27]:
class_weights = nu.getWeights(trainy)
class_weights

{0: 1.0598192185017794, 1: 0.9465727358718845}

In [28]:
#neural network
model = Sequential()

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

opt = Adam(lr=1e-3, decay=1e-5)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
callbacks = []

#stop model from training when it starts to get bad
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                min_delta=0, 
                                patience=5, 
                                verbose=0, 
                                mode='auto', 
                                baseline=None, 
                                restore_best_weights=True)
callbacks.append(early_stop)


# # getting the model_version name for the model checkpoint callback
# # keep is_nn to false here so it doesn't save the model just yet
# version_number = nu.exportModel(model, model_name, False, indicators, features, proba_threshold=0.0, is_nn=False, save_model=False)

# #saving model when validation accuracy gets better
# checkpoint_call = tf.keras.callbacks.ModelCheckpoint(f'./v2/strategy/saved_models/{model_name}/{version_number}', 
#                                      monitor='val_accuracy', 
#                                      verbose=0, 
#                                      save_best_only=True,
#                                      save_weights_only=False, 
#                                      mode='auto', 
#                                      save_freq='epoch')
# callbacks.append(checkpoint_call)

# if not os.path.isdir(f'./v2/strategy/saved_models/{model_name}/{version_number}/logs'):
#     os.mkdir(f'./v2/strategy/saved_models/{model_name}/{version_number}/logs')
# tensorb = tf.keras.callbacks.TensorBoard(log_dir=f'./v2/strategy/saved_models/{model_name}/{version_number}/logs',
#                                  histogram_freq=0, 
#                                  write_graph=True, 
#                                  write_images=False,    
#                                  update_freq='epoch', 
#                                  profile_batch=2, 
#                                  embeddings_freq=0,    
#                                  embeddings_metadata=None)
# callbacks.append(tensorb)

In [30]:
model.fit(trainX.drop("close", axis=1).values, trainy.values, callbacks=callbacks, epochs=30, class_weight=class_weights, validation_data=(testX.drop('close', axis=1).values, testy.values))

Epoch 1/30
5379/5379 [==============================] - 8s 2ms/step - loss: 0.7024 - accuracy: 0.5386 - val_loss: 0.6762 - val_accuracy: 0.5859
Epoch 2/30
5379/5379 [==============================] - 8s 1ms/step - loss: 0.6798 - accuracy: 0.5680 - val_loss: 0.6827 - val_accuracy: 0.5644
Epoch 3/30
5379/5379 [==============================] - 8s 2ms/step - loss: 0.6768 - accuracy: 0.5768 - val_loss: 0.6804 - val_accuracy: 0.5571
Epoch 4/30
5379/5379 [==============================] - 8s 2ms/step - loss: 0.6748 - accuracy: 0.5809 - val_loss: 0.6768 - val_accuracy: 0.5712
Epoch 5/30
5379/5379 [==============================] - 8s 2ms/step - loss: 0.6731 - accuracy: 0.5832 - val_loss: 0.6728 - val_accuracy: 0.5831
Epoch 6/30
5379/5379 [==============================] - 9s 2ms/step - loss: 0.6725 - accuracy: 0.5855 - val_loss: 0.6855 - val_accuracy: 0.5498
Epoch 7/30
5379/5379 [==============================] - 8s 2ms/step - loss: 0.6719 - accuracy: 0.5861 - val_loss: 0.6766 - val_accuracy:

In [32]:
model_version = nu.exportModel(model, model_name, new_version=False, indicators=indicators, features=features, proba_threshold=0.7, is_nn=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./v2/strategy/saved_models/gamemaker/1_1/assets
